### Set the stage

In [1]:
import sys
from datetime import date
sys.path.append('src/')
from finance.common.dao import Dao
from finance.common.model import Position
from finance.common.calculate import CalcPosition

def get_portfolio(portfolio_name, portfolio_allocation):
        closings = []
        targets = []
        positions = {}
        for instrument_id, instrument_name, percentage in portfolio_allocation:
            positions[instrument_id] = Position(instrument_id, instrument_name)
            quote = dao.get_stock_latest_quotes(date.today(), instrument_name, instrument_id)
            closings.append((instrument_name, quote[instrument_id].price))
            targets.append((instrument_name, percentage))
        return closings, targets, positions

name = 'Monthly ETF'

dao = Dao('finance.db')
dao.connect()

closings, targets, positions = get_portfolio(name, dao.get_portfolios(name)[name])
calc_pos = CalcPosition(date.today(), positions)
calc_pos.calc(dao)
positions = [[p.name, p.shares] for n, p in positions.items()]

dao.close()

URL root at /finance_demo


In [2]:
import pandas as pd
from IPython.display import display

positions = pd.DataFrame(columns=['ticker', 'shares'], data=positions)
closing = pd.DataFrame(columns=['ticker', 'price'], data=closings)
target = pd.DataFrame(columns=['ticker', 'target_allocation'], data=targets)

In [3]:
from functools import reduce
my_portfolio = reduce(lambda left,right: pd.merge(left,right,on='ticker'), [positions, closing, target])

def calc_cur_allocation(portfolio):
    portfolio['market_value'] = portfolio['shares']*portfolio['price']
    total_value = portfolio.market_value.sum()
    portfolio['current_allocation'] = 100 * portfolio['market_value'] / total_value

calc_cur_allocation(my_portfolio)
display(my_portfolio)

,ticker,shares,price,target_allocation,market_value,current_allocation
0,VTI,87.0,149.029999,47.0,12965.609913,58.754295
1,VXUS,103.0,51.299999,34.0,5283.899897,23.944251
2,VNQ,14.0,93.070000,5.0,1302.980000,5.904518
3,VOO,6.0,269.320007,10.0,1615.920042,7.322621
4,VBK,5.0,179.820007,4.0,899.100035,4.074316


In [4]:
print(my_portfolio.to_dict('records'))

[{'ticker': 'VTI', 'shares': 87.0, 'price': 149.029999, 'target_allocation': 47.0, 'market_value': 12965.609913, 'current_allocation': 58.754295248500426}, {'ticker': 'VXUS', 'shares': 103.0, 'price': 51.299999, 'target_allocation': 34.0, 'market_value': 5283.899897, 'current_allocation': 23.944250729044665}, {'ticker': 'VNQ', 'shares': 14.0, 'price': 93.07, 'target_allocation': 5.0, 'market_value': 1302.98, 'current_allocation': 5.904517576618772}, {'ticker': 'VOO', 'shares': 6.0, 'price': 269.320007, 'target_allocation': 10.0, 'market_value': 1615.9200419999997, 'current_allocation': 7.322620677523478}, {'ticker': 'VBK', 'shares': 5.0, 'price': 179.820007, 'target_allocation': 4.0, 'market_value': 899.100035, 'current_allocation': 4.074315768312678}]


In [5]:
def rebalance_without_new_fund(portfolio):
    total = portfolio.market_value.sum()
    portfolio["target_market_value"] = total * portfolio["target_allocation"]/100
    portfolio["fund_to_transfer"] = portfolio["target_market_value"] - portfolio["market_value"]
    portfolio["delta_shares"] = round(portfolio["fund_to_transfer"] /portfolio["price"])
    portfolio["shares"] =  portfolio["shares"] + portfolio["delta_shares"] 
    
    calc_cur_allocation(portfolio)

In [6]:
rebalance_without_new_fund(my_portfolio)
rebalancing_step1 = my_portfolio.copy()
display(my_portfolio)
print("Sum transfered fund {}".format(my_portfolio.fund_to_transfer.sum()))

,ticker,shares,price,target_allocation,market_value,current_allocation,target_market_value,fund_to_transfer,delta_shares
0,VTI,70.0,149.029999,47.0,10432.099930,47.220311,10371.729647,-2593.880266,-17.0
1,VXUS,146.0,51.299999,34.0,7489.799854,33.902156,7502.953362,2219.053465,43.0
2,VNQ,12.0,93.070000,5.0,1116.840000,5.055313,1103.375494,-199.604506,-2.0
3,VOO,8.0,269.320007,10.0,2154.560056,9.752494,2206.750989,590.830947,2.0
4,VBK,5.0,179.820007,4.0,899.100035,4.069726,882.700395,-16.399640,-0.0


Sum transfered fund -3.183231456205249e-12


In [7]:
def reblance_with_new_fund(portfolio, fund):
    portfolio["fund_allocation"] = fund * portfolio["target_allocation"] / 100
    portfolio["delta_shares"] = round(portfolio["fund_allocation"] /portfolio["price"])
    portfolio["shares"] += portfolio["delta_shares"]
    calc_cur_allocation(portfolio)

reblance_with_new_fund(my_portfolio,1000)
display(my_portfolio)

,ticker,shares,price,target_allocation,market_value,current_allocation,target_market_value,fund_to_transfer,delta_shares,fund_allocation
0,VTI,73.0,149.029999,47.0,10879.189927,47.317984,10371.729647,-2593.880266,3.0,470.0
1,VXUS,153.0,51.299999,34.0,7848.899847,34.138030,7502.953362,2219.053465,7.0,340.0
2,VNQ,13.0,93.070000,5.0,1209.910000,5.262386,1103.375494,-199.604506,1.0,50.0
3,VOO,8.0,269.320007,10.0,2154.560056,9.371050,2206.750989,590.830947,0.0,100.0
4,VBK,5.0,179.820007,4.0,899.100035,3.910549,882.700395,-16.399640,0.0,40.0


In [8]:
my_portfolio["delta_shares"] += rebalancing_step1["delta_shares"]
display(my_portfolio)

,ticker,shares,price,target_allocation,market_value,current_allocation,target_market_value,fund_to_transfer,delta_shares,fund_allocation
0,VTI,73.0,149.029999,47.0,10879.189927,47.317984,10371.729647,-2593.880266,-14.0,470.0
1,VXUS,153.0,51.299999,34.0,7848.899847,34.138030,7502.953362,2219.053465,50.0,340.0
2,VNQ,13.0,93.070000,5.0,1209.910000,5.262386,1103.375494,-199.604506,-1.0,50.0
3,VOO,8.0,269.320007,10.0,2154.560056,9.371050,2206.750989,590.830947,2.0,100.0
4,VBK,5.0,179.820007,4.0,899.100035,3.910549,882.700395,-16.399640,0.0,40.0


In [9]:
new_trades = (('VXUS', 52),)
port = my_portfolio.copy()
port['delta_shares'] = 0
for ticker, shares in new_trades:
    row = port['ticker'] == ticker
    port.loc[row, 'shares'] += shares
    port.loc[row, 'delta_shares'] = shares

calc_cur_allocation(port)
display(port)

,ticker,shares,price,target_allocation,market_value,current_allocation,target_market_value,fund_to_transfer,delta_shares,fund_allocation
0,VTI,73.0,149.029999,47.0,10879.189927,42.398690,10371.729647,-2593.880266,0,470.0
1,VXUS,205.0,51.299999,34.0,10516.499795,40.985203,7502.953362,2219.053465,52,340.0
2,VNQ,13.0,93.070000,5.0,1209.910000,4.715296,1103.375494,-199.604506,0,50.0
3,VOO,8.0,269.320007,10.0,2154.560056,8.396813,2206.750989,590.830947,0,100.0
4,VBK,5.0,179.820007,4.0,899.100035,3.503998,882.700395,-16.399640,0,40.0
